# 🔍 Information Retrieval System using TF-IDF, Word2Vec, BERT, Hybrid

This notebook presents a full pipeline for building and evaluating an Information Retrieval (IR) system using different vector representations and integrating a generative component (RAG).

## 📦 Step 1: Imports and Utilities

In [46]:
import os
os.chdir("D:/retrieval_project")
print("Current dir:", os.getcwd())


Current dir: D:\retrieval_project


In [47]:
import sys

# إضافة src إلى sys.path
src_path = os.path.join(os.getcwd(), "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print("✅ src added to sys.path:", src_path)


✅ src added to sys.path: D:\retrieval_project\src


## 🧹 Step 2: Data Preprocessing

We clean the raw dataset using tokenization, stopword removal, and lemmatization.

In [32]:
!python src/preprocess_dataset.py

🚀 بدء المعالجة المسبقة للبيانات...
✅ تم حفظ الملفات المنظفة بنجاح:
📄 data/antique\cleaned_corpus.jsonl
📄 data/antique\cleaned_queries.jsonl



🔄 Cleaning corpus.jsonl: 0it [00:00, ?it/s]
🔄 Cleaning corpus.jsonl: 1it [00:04,  4.63s/it]
🔄 Cleaning corpus.jsonl: 251it [00:04, 75.03it/s]
🔄 Cleaning corpus.jsonl: 437it [00:04, 143.91it/s]
🔄 Cleaning corpus.jsonl: 575it [00:05, 210.21it/s]
🔄 Cleaning corpus.jsonl: 846it [00:05, 388.22it/s]
🔄 Cleaning corpus.jsonl: 1098it [00:05, 584.70it/s]
🔄 Cleaning corpus.jsonl: 1389it [00:05, 853.70it/s]
🔄 Cleaning corpus.jsonl: 1660it [00:05, 1116.62it/s]
🔄 Cleaning corpus.jsonl: 1903it [00:05, 1336.31it/s]
🔄 Cleaning corpus.jsonl: 2146it [00:05, 1214.24it/s]
🔄 Cleaning corpus.jsonl: 2442it [00:05, 1523.41it/s]
🔄 Cleaning corpus.jsonl: 2725it [00:05, 1785.24it/s]
🔄 Cleaning corpus.jsonl: 2979it [00:06, 1950.54it/s]
🔄 Cleaning corpus.jsonl: 3234it [00:06, 2091.19it/s]
🔄 Cleaning corpus.jsonl: 3494it [00:06, 2216.48it/s]
🔄 Cleaning corpus.jsonl: 3784it [00:06, 2393.42it/s]
🔄 Cleaning corpus.jsonl: 4082it [00:06, 2548.88it/s]
🔄 Cleaning corpus.jsonl: 4354it [00:06, 2583.21it/s]
🔄 Cleaning corpus

## 🔠 Step 3: Build Representations

In [ ]:
!python representation.py

💾 [TF-IDF] حفظ النموذج والـ matrix ...
✅ [TF-IDF] تم تمثيل الوثائق بنجاح.
(ir_env) PS D:\retrieval_project> python src/representation.py
🔍 [TF-IDF] تحميل الوثائق المنظفة...
⚙️ [TF-IDF] تدريب نموذج TF-IDF ...
D:\retrieval_project\ir_env\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
💾 [TF-IDF] حفظ النموذج والـ matrix ...
✅ [TF-IDF] تم تمثيل الوثائق بنجاح.
D:\retrieval_project\ir_env\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
💾 [TF-IDF] حفظ النموذج والـ matrix ...
✅ [TF-IDF] تم تمثيل الوثائق بنجاح.
  warnings.warn(
💾 [TF-IDF] حفظ النموذج والـ matrix ...
✅ [TF-IDF] تم تمثيل الوثائق بنجاح.
✅ [TF-IDF] تم تمثيل الوثائق بنجاح.
🔤 [W2V] تحميل الوثائق المنظفة ...
⚙️ [W2V] تدريب نموذج Word2Vec ...
📊 [W2V] إنشاء تمثيلات الوثائق ...
⚙️ [W2V] تدريب نموذج Word2Vec ...
📊 [W2V] إنشاء تمثيلات الوثائق ...
📊 [W2V] إنشاء تمثيلات الوثائق ...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 403666/403666 [00:38<00:00, 10574.89it/s] 
💾 [W2V] حفظ النموذج والتمثيلات والـ vectorizer...
✅ [W2V] تم تمثيل الوثائق بنجاح.
🤖 [BERT] تحميل الوثائق المنظفة ...
📦 [BERT] تحميل النموذج: sentence-transformers/all-MiniLM-L6-v2
📊 [BERT] إنشاء تمثيلات الوثائق ...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 403666/403666 [2:21:05<00:00, 47.68it/s]   
💾 [BERT] حفظ التمثيلات ...
💾 [BERT] حفظ vectorizer (tokenizer + model) ...
✅ [BERT] تم تمثيل الوثائق بنجاح.
🧬 [Hybrid] تحميل تمثيلات TF-IDF و BERT ...
🔗 [Hybrid] دمج التمثيلين ...
💾 [Hybrid] حفظ التمثيلات المدمجة ...
✅ [Hybrid] تم بناء التمثيل الهجين بنجاح.

## 🧭 Step 4: Build Indexes

In [33]:
!python src/indexing.py

📦 تحميل التمثيلات من vector_stores\quora_tfidf_matrix.joblib ...
🔍 Sparse تمثيل، اختيار algorithm مناسب...
⚙️ بناء فهرس باستخدام NearestNeighbors (n=10) ...
💾 حفظ الفهرس إلى vector_stores\quora_tfidf_index.joblib ...
✅ تم بناء الفهرس بنجاح.
📦 تحميل التمثيلات من vector_stores\quora_word2vec_vectors.joblib ...
🔍 Dense تمثيل، اختيار algorithm مناسب...
⚙️ بناء فهرس باستخدام NearestNeighbors (n=10) ...
💾 حفظ الفهرس إلى vector_stores\quora_word2vec_index.joblib ...
✅ تم بناء الفهرس بنجاح.
📦 تحميل التمثيلات من vector_stores\quora_bert_vectors.joblib ...
🔍 Dense تمثيل، اختيار algorithm مناسب...
⚙️ بناء فهرس باستخدام NearestNeighbors (n=10) ...
💾 حفظ الفهرس إلى vector_stores\quora_bert_index.joblib ...
✅ تم بناء الفهرس بنجاح.
📦 تحميل التمثيلات من vector_stores\quora_hybrid_vectors.joblib ...
🔍 Sparse تمثيل، اختيار algorithm مناسب...
⚙️ بناء فهرس باستخدام NearestNeighbors (n=10) ...
💾 حفظ الفهرس إلى vector_stores\quora_hybrid_index.joblib ...
✅ تم بناء الفهرس بنجاح.


## 🔍 Step 5: Retrieve Top-K Documents

In [49]:
import os
os.chdir("D:/retrieval_project")
print("📁 Working directory set to:", os.getcwd())


📁 Working directory set to: D:\retrieval_project


In [59]:
from preprocessing import clean_text

def custom_tokenizer(text):
    return clean_text(text).split()

import joblib
import json
from sklearn.neighbors import NearestNeighbors
from retrieval import retrieve_top_k_index
from scipy.sparse import csr_matrix

dataset = "antique"
representation = "hybrid"

vectorizer_path = f"vector_stores/{dataset}_tfidf_vectorizer.joblib"
bert_vectorizer_path = f"vector_stores/{dataset}_bert_vectorizer.joblib"
vectors_path = f"vector_stores/{dataset}_{representation}_vectors.joblib"
corpus_path = f"data/{dataset}/cleaned_corpus.jsonl"

# تحميل الـ vectorizer
tfidf_vectorizer = joblib.load(vectorizer_path)
tokenizer, model = joblib.load(bert_vectorizer_path)
vectorizer = (tfidf_vectorizer, tokenizer, model)

# تحميل التمثيلات
doc_ids, vectors = joblib.load(vectors_path, mmap_mode='r')

# ✅ تحويل المصفوفة إلى csr_matrix لتمكين التقطيع
if hasattr(vectors, "tocsc"):  # إذا كانت sparse
    vectors = vectors.tocsr()

# ✅ تقليل عدد الوثائق مؤقتًا لتجنب MemoryError
doc_ids = doc_ids[:3000]
vectors = vectors[:3000]

# بناء الفهرس
index = NearestNeighbors(n_neighbors=10, metric='cosine')
index.fit(vectors)

# تحميل البيانات النصية
corpus = {json.loads(line)["_id"]: json.loads(line) for line in open(corpus_path, encoding='utf-8')}

# اختبار الاسترجاع
results = retrieve_top_k_index("information about search engine", vectorizer, index, doc_ids, corpus)
results[:3]


[{'doc_id': '176362_1', 'text': 'google', 'score': 0.5889042271128784},
 {'doc_id': '4327259_0',
  'text': 'may want try infopleasecom check meaning term',
  'score': 0.4406538979945398},
 {'doc_id': '4303179_3',
  'text': 'try google search miserable failure look first result',
  'score': 0.41501893238097776}]

## 📊 Step 6: Evaluate Representations for quora

In [38]:
!python src/evaluation.py

⏩ Skipping TFIDF (already evaluated)
⏩ Skipping WORD2VEC (already evaluated)
⏩ Skipping BERT (already evaluated)
⏩ Skipping HYBRID (already evaluated)

📊 Final Evaluation Results:
╒═══════════╤══════════════════╤════════╤════════╤════════╤═════════════╕
│ Dataset   │ Representation   │    MAP │    MRR │   P@10 │   Recall@10 │
╞═══════════╪══════════════════╪════════╪════════╪════════╪═════════════╡
│ QUORA     │ TFIDF            │ 0.4366 │ 0.4649 │ 0.0719 │      0.5405 │
├───────────┼──────────────────┼────────┼────────┼────────┼─────────────┤
│ QUORA     │ WORD2VEC         │ 0.3001 │ 0.3278 │ 0.0501 │      0.3835 │
├───────────┼──────────────────┼────────┼────────┼────────┼─────────────┤
│ QUORA     │ BERT             │ 0.5048 │ 0.5289 │ 0.0826 │      0.5993 │
├───────────┼──────────────────┼────────┼────────┼────────┼─────────────┤
│ QUORA     │ HYBRID           │ 0.5057 │ 0.5297 │ 0.0827 │      0.6001 │
╘═══════════╧══════════════════╧════════╧════════╧════════╧═════════════╛


## Evaluate Representations for antique

In [39]:
!python src/evaluation.py

⏩ Skipping TFIDF (already evaluated)
⏩ Skipping WORD2VEC (already evaluated)
⏩ Skipping BERT (already evaluated)
⏩ Skipping HYBRID (already evaluated)

📊 Final Evaluation Results:
╒═══════════╤══════════════════╤════════╤════════╤════════╤═════════════╕
│ Dataset   │ Representation   │    MAP │    MRR │   P@10 │   Recall@10 │
╞═══════════╪══════════════════╪════════╪════════╪════════╪═════════════╡
│ ANTIQUE   │ TFIDF            │ 0.0965 │ 0.716  │ 0.362  │      0.1214 │
├───────────┼──────────────────┼────────┼────────┼────────┼─────────────┤
│ ANTIQUE   │ WORD2VEC         │ 0.0363 │ 0.4539 │ 0.173  │      0.0534 │
├───────────┼──────────────────┼────────┼────────┼────────┼─────────────┤
│ ANTIQUE   │ BERT             │ 0.0956 │ 0.7544 │ 0.3715 │      0.1214 │
├───────────┼──────────────────┼────────┼────────┼────────┼─────────────┤
│ ANTIQUE   │ HYBRID           │ 0.096  │ 0.7608 │ 0.376  │      0.123  │
╘═══════════╧══════════════════╧════════╧════════╧════════╧═════════════╛


## 🧠 Step 7: RAG - Generate Answers

Use top retrieved results as context to a generative model.

In [60]:
from app import generate_rag_answer

query = "What is vector space model?"
context = [r["text"] for r in results[:5]]
generate_rag_answer(query, context)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


'vector space model'